In [1]:
# make SLDB datasets for BSCTRFM from individual time series

# first, use this code for trimmed datasets only
# later, generalize for all the time series in the electricity dataset

In [2]:
import numpy as np
import pandas as pd

In [3]:
import json

In [4]:
import os

In [5]:
import tensorflow as tf

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import joblib

In [7]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.layouts import row, gridplot, layout
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [8]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [9]:
# the main source is the electricity dataset LD2011-2014 from UCI

In [10]:
# it resides in
dataset_path = '/home/developer/gcp/cbidmltsf/datasets/electricity'

In [11]:
os.listdir(dataset_path)

['LD2011_2014.txt',
 'separated_preprocessed',
 'separated_raw',
 'hourly_electricity_complete.pkl',
 'hourly_electricity.csv',
 'LD2011_2014.txt.zip',
 'hourly_electricity_filtered_academic_papers.pkl']

In [12]:
# 'LD2011_2014.txt'                                          source from UCI
# 'LD2011_2014.txt.zip'                                      source from UCI, compressed
# 'hourly_electricity.csv'                                   complete dataset in CSV
# 'hourly_electricity_complete.pkl'                          complete dataset in Pandas
# 'hourly_electricity_filtered_academic_papers.pkl'          filtered dataset for benchmarking
# 'separated_raw/'                                           pickles per customer, raw data
# 'separated_preprocessed/'                                  pickles per customer, outliers removed

In [13]:
# a SLDB is produced from separated time series (raw or preprocessed)

# SLDB contents are:
# TFRecord files for training
# TFRecord files for evaluation (if eval required)
# time series pickles for testing

In [14]:
# constant values for positional encodings
hours_in_day = 24
days_in_week = 7
days_in_month = 30
days_in_year = 365

In [15]:
# a constant to make sin/cos functions from hours_from_start (the 'age' covariate)
total_hours = 32303

In [16]:
# define global dataset intervals (they might not be precise when missing values exist)

# split the time series in seen (train, eval) and unseen (test) data
# according to academic papers:

# 243 days on seen data, 7 days on unseen data 

# seen data:      '2014-01-01 00:00:00' to '2014-08-31 23:00:00', 243*24 = 5832 lectures

# train/eval split is 0.9/0.1, then

# train data:     '2014-01-01 00:00:00' to '2014-08-07 15:00:00', 5248 lectures
# eval data:      '2014-08-07 16:00:00' to '2014-08-31 23:00:00', 584 lectures

# unseen data:    '2014-09-01 00:00:00' to '2014-09-07 23:00:00', 7*24 = 168 lectures

dates = {
    'train': {
        'start': '2014-01-01 00:00:00',
        'end': '2014-08-07 15:00:00',
    },
    'eval': {
        'start': '2014-08-07 16:00:00',
        'end': '2014-08-31 23:00:00',
    },
    'test': {
        'start': '2014-09-01 00:00:00',
        'end': '2014-09-07 23:00:00',
    },
}

In [17]:
# build sub-series to be persisted as serialized training examples

# dimensionality of the encoder input
m = 168

# dimensionality of the decoder output 
t = 168

In [18]:
# columns to be included in the SLDB

# use 7D encoder (age, hour-day, day-week)

sldb_columns = [
    'date',
    'token_id',
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'    
]

In [19]:
sldb = {
    'ts': 'LD2011-2014_SEPARATED_FULL',
    'embedding': {
        'hourly': 168
    },
    'tau': {
        'hourly': 1
    },
    'no_targets': 168,
    'BSCTRFM': 1,
    'preprocessed': 0
}

In [20]:
sldb

{'ts': 'LD2011-2014_SEPARATED_FULL',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'preprocessed': 0}

In [21]:
# a string with the basic specifications of the SLDB, as part of the SLDB identifier

# add the suffix '11D' to differentiate this SLDB from the original one, which is 9D

# add the suffix MMX to indicate the scaler used was MinMax
# add the suffix STD to indicate the scaler used was Standard

sldb_specs = 'BSCTRFM_{:03d}_{:03d}_07DB_MMX'.format(sldb['embedding']['hourly'], sldb['no_targets'])
sldb_specs

'BSCTRFM_168_168_07DB_MMX'

In [22]:
# get the time-based identifier for the SLDB
sldb_identifier = '{}_{}'.format(sldb['ts'], sldb_specs)
sldb_identifier

'LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [23]:
sldb_dir = '/home/developer/gcp/cbidmltsf/sldbs/{}'.format(sldb_identifier)
sldb_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [24]:
# get a path to the scalers sub-directory
scalers_dir = '{}/scalers'.format(sldb_dir)
scalers_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX/scalers'

In [25]:
# DO NOT CREATE SLDB FOLDERS, THEY WERE CREATED BEFORE

In [26]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'
]

In [27]:
# both the encoder input and the decoder input use the same columns from the source sub_series dataframe
decoder_input_columns = encoder_input_columns

In [28]:
target_columns = ['kw_scaled']

In [29]:
id_columns = ['token_id']

In [30]:
# a dictionary to manage data per individual customer_id
data = dict()

In [31]:
# a dictionary to store the number of examples per customer_id, stage
count = dict()

In [32]:
# the number of cores available for training in Cloud TPU
num_cores = 8

In [68]:
# there is no data at all in MT-223, then use MT-224 time series as a base template
token_ids = [223, 224]

In [69]:
customer_ids = ['MT_{:03d}'.format(token_id) for token_id in token_ids]

In [70]:
# are we training over raw data or preprocessed data?
state = 'raw'

In [71]:
for customer_id in customer_ids:
    customer_data_path = '{}/separated_{}/{}.pkl'.format(dataset_path, state, customer_id)
    data[customer_id] = pd.read_pickle(customer_data_path)

In [72]:
data['MT_223']

,power_usage,token_id,date,hours_from_start,days_from_start,hour_of_day,day_of_week,day_of_month,day_of_year,week_of_year,month_of_year


In [73]:
# make a copy for the zero-value dataset
data['MT_223'] = data['MT_224'].copy()

In [74]:
# adjust the required column values
data['MT_223']['token_id'] = 223
data['MT_223']['power_usage'] = 0.0

In [75]:
data['MT_223']

,power_usage,token_id,date,hours_from_start,days_from_start,hour_of_day,day_of_week,day_of_month,day_of_year,week_of_year,month_of_year
5817703,0.0,223,2014-01-01 00:00:00,26304.0,1096,0,2,1,1,1,1
5817704,0.0,223,2014-01-01 01:00:00,26305.0,1096,1,2,1,1,1,1
5817705,0.0,223,2014-01-01 02:00:00,26306.0,1096,2,2,1,1,1,1
5817706,0.0,223,2014-01-01 03:00:00,26307.0,1096,3,2,1,1,1,1
5817707,0.0,223,2014-01-01 04:00:00,26308.0,1096,4,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
5823698,0.0,223,2014-09-07 19:00:00,32299.0,1345,19,6,7,250,36,9
5823699,0.0,223,2014-09-07 20:00:00,32300.0,1345,20,6,7,250,36,9
5823700,0.0,223,2014-09-07 21:00:00,32301.0,1345,21,6,7,250,36,9
5823701,0.0,223,2014-09-07 22:00:00,32302.0,1345,22,6,7,250,36,9


In [76]:
# a dictionary for trimming dates per customer_id

# use all zero-value time series for MT-223
train_start_date = {'MT_223': '2014-01-01 00:00:00'}

In [77]:
# now break the main loop and produce the SLDB for MT-223 using separated code cells

In [79]:
customer_id = 'MT_223'

In [80]:
# initialize the examples dictionary for each customer
examples = {
    'train': [],
    'eval': [],
    # test dataset is not passed to SLDB
    # 'test': []
}

In [81]:
# use now a reference to the dataframe in the data dictionary 
data_df = data[customer_id]

In [83]:
# a sub-dictionary to keep the number of examples per customer_id, stage
count[customer_id] = dict()

In [84]:
# expand with positional encodings
data_df['sin_hours_from_start'] = np.sin(2*np.pi*data_df.hours_from_start/total_hours)
data_df['cos_hours_from_start'] = np.cos(2*np.pi*data_df.hours_from_start/total_hours)
data_df['sin_hour_day'] = np.sin(2*np.pi*data_df.hour_of_day/hours_in_day)
data_df['cos_hour_day'] = np.cos(2*np.pi*data_df.hour_of_day/hours_in_day)
data_df['sin_day_week'] = np.sin(2*np.pi*data_df.day_of_week/days_in_week)
data_df['cos_day_week'] = np.cos(2*np.pi*data_df.day_of_week/days_in_week)

In [86]:
# get the time series indexes that delimit train, eval, and test intervals

# train interval goes from the first available lecture (ideally '2014-01-01 00:00:00')
# to '2014-08-07 15:00:00' (ideally 5248 lectures)
train_start_index = data_df[data_df['date'] == pd.to_datetime(train_start_date[customer_id])].index[0]
train_end_index = data_df[data_df['date'] == pd.to_datetime(dates['train']['end'])].index[0]
print('{} train interval: from {} on {} to {} on {}, {} lectures'.\
     format(customer_id,
            train_start_index, train_start_date[customer_id],
            train_end_index, dates['train']['end'],
            train_end_index - train_start_index + 1))

MT_223 train interval: from 5817703 on 2014-01-01 00:00:00 to 5822950 on 2014-08-07 15:00:00, 5248 lectures


In [87]:
eval_start_index = data_df[data_df['date'] == pd.to_datetime(dates['eval']['start'])].index[0]
eval_end_index = data_df[data_df['date'] == pd.to_datetime(dates['eval']['end'])].index[0]
print('{} eval interval: from {} on {} to {} on {}, {} lectures'.\
     format(customer_id,
            eval_start_index, dates['eval']['start'],
            eval_end_index, dates['eval']['end'],
            eval_end_index - eval_start_index + 1))

MT_223 eval interval: from 5822951 on 2014-08-07 16:00:00 to 5823534 on 2014-08-31 23:00:00, 584 lectures


In [89]:
# get a series for the power usage variable on the training dataset, to fit the scaler
# set up the upper limit of this series based on a fixed date, not on a fixed value!!!
lectures_train_data = data_df['power_usage'].loc[train_start_index:train_end_index]
lectures_train_data

5817703    0.0
5817704    0.0
5817705    0.0
5817706    0.0
5817707    0.0
          ... 
5822946    0.0
5822947    0.0
5822948    0.0
5822949    0.0
5822950    0.0
Name: power_usage, Length: 5248, dtype: float64

In [90]:
# fit a scaler only on train data
# it is required to pass the power usage time series to a (?, 1) NumPy array
lectures_train_data_array = np.array(lectures_train_data).reshape(-1, 1)

In [91]:
# get MinMaxScaler on train data, store it in a dictionary
scaler_type = 'min_max'
scaler = MinMaxScaler()
fitted_scaler = scaler.fit(lectures_train_data_array)
print('Scaler {} generated on training data for {}'.format(scaler_type, customer_id))

Scaler min_max generated on training data for MT_223


In [92]:
# persist the scaler
scaler_filename = '{}/{}_{}.save'.format(scalers_dir, scaler_type, customer_id)
joblib.dump(fitted_scaler, scaler_filename)
print('Scaler {} persisted for {}'.format(scaler_type, customer_id))

Scaler min_max persisted for MT_223


In [93]:
# get an array from the variable time series (seen and unseen)
all_data_variable_array = np.array(data_df.power_usage).reshape(-1, 1)

In [94]:
# apply the scaler over all data (seen and unseen)
# rescale, and squeeze to drop the extra dimension, then assign to the new column kw_scaled
data_df['kw_scaled'] = np.squeeze(fitted_scaler.transform(all_data_variable_array))

In [95]:
# at this moment, the individual time series are ready to be window-rolled to produce
# sub-series/examples to serialize

# BSCTRFM inference process is not direct, but iterative, therefore
# no TFRecord SLDB is required for test dataset,

test_start_index = data_df[data_df['date'] == pd.to_datetime(dates['test']['start'])].index[0]
test_end_index = data_df[data_df['date'] == pd.to_datetime(dates['test']['end'])].index[0]

In [96]:
# the time series used to build the test dataset must go
# from '2014-08-18 01:00:00' to '2014-09-07 23:00:00'
# in order to extract 168 features with targets
# (the last element in the decoder output)
# ranging from '2014-09-01 00:00:00' to '2014-09-07 23:00:00'

# therefore
test_ts_start_index = data_df[data_df['date'] == pd.to_datetime('2014-08-18 01:00:00')].index[0]

In [97]:
# persist only the time series corresponding to the inference interval as test dataset
test_time_series = data_df[sldb_columns].loc[test_ts_start_index:test_end_index]

print('{} test interval: from {} on {} to {} on {}, {} lectures'.\
     format(customer_id,
            test_ts_start_index, '2014-08-18 01:00:00',
            test_end_index, dates['test']['end'],
            test_end_index - test_ts_start_index + 1))

MT_223 test interval: from 5823200 on 2014-08-18 01:00:00 to 5823702 on 2014-09-07 23:00:00, 503 lectures


In [98]:
test_time_series

,date,token_id,kw_scaled,sin_hours_from_start,cos_hours_from_start,sin_hour_day,cos_hour_day,sin_day_week,cos_day_week
5823200,2014-08-18 01:00:00,223,0.0,-9.748783e-02,0.995237,0.258819,0.965926,0.000000,1.00000
5823201,2014-08-18 02:00:00,223,0.0,-9.729425e-02,0.995256,0.500000,0.866025,0.000000,1.00000
5823202,2014-08-18 03:00:00,223,0.0,-9.710066e-02,0.995275,0.707107,0.707107,0.000000,1.00000
5823203,2014-08-18 04:00:00,223,0.0,-9.690707e-02,0.995293,0.866025,0.500000,0.000000,1.00000
5823204,2014-08-18 05:00:00,223,0.0,-9.671348e-02,0.995312,0.965926,0.258819,0.000000,1.00000
...,...,...,...,...,...,...,...,...,...
5823698,2014-09-07 19:00:00,223,0.0,-7.780311e-04,1.000000,-0.965926,0.258819,-0.781831,0.62349
5823699,2014-09-07 20:00:00,223,0.0,-5.835234e-04,1.000000,-0.866025,0.500000,-0.781831,0.62349
5823700,2014-09-07 21:00:00,223,0.0,-3.890156e-04,1.000000,-0.707107,0.707107,-0.781831,0.62349
5823701,2014-09-07 22:00:00,223,0.0,-1.945078e-04,1.000000,-0.500000,0.866025,-0.781831,0.62349


In [99]:
# path to persist the time series dataframe corresponding to test dataset
path = '{}/test/{}.pkl'.format(sldb_dir, customer_id)

test_time_series.to_pickle(path)
print('Test dataset persisted as a time series pickle for {}'.format(customer_id))

Test dataset persisted as a time series pickle for MT_223


In [100]:
# make SLDB training dataset
# get an iterable with all the possible sub-series for training examples
train_starting_indexes = np.arange(train_start_index, train_end_index - (m + t) + 2)

for train_starting_index in train_starting_indexes:

    # substract 1 at the end of the slice because loc works different from direct slicing!
    sub_series_df = data_df[sldb_columns].loc[train_starting_index:train_starting_index + (m + t) - 1]

    encoder_input_df = sub_series_df[encoder_input_columns][:m]
    decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
    target_df = sub_series_df[target_columns][m:m+t]
    id_df = sub_series_df[id_columns][:1]

    encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
    decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
    target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
    id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

    examples['train'].append(
        {
            'encoder_input': encoder_input_list,
            'decoder_input': decoder_input_list,
            'target': target_list,
            'id': id_list,
        }
    )

print('{} processed. The number of examples in {} dataset is {}'.\
      format(customer_id, 'train', len(examples['train'])))

MT_223 processed. The number of examples in train dataset is 4913


In [101]:
# make SLDB evaluation dataset
# get an iterable with all the possible sub-series for evaluation examples
eval_starting_indexes = np.arange(eval_start_index, eval_end_index - (m + t) + 2)

for eval_starting_index in eval_starting_indexes:

    # substract 1 at the end of the slice because loc works different from direct slicing!
    sub_series_df = data_df[sldb_columns].loc[eval_starting_index:eval_starting_index + (m + t) - 1]

    encoder_input_df = sub_series_df[encoder_input_columns][:m]
    decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
    target_df = sub_series_df[target_columns][m:m+t]
    id_df = sub_series_df[id_columns][:1]

    encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
    decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
    target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
    id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

    examples['eval'].append(
        {
            'encoder_input': encoder_input_list,
            'decoder_input': decoder_input_list,
            'target': target_list,
            'id': id_list,
        }
    )

print('{} processed. The number of examples in {} dataset is {}'.\
      format(customer_id, 'eval', len(examples['eval'])))

MT_223 processed. The number of examples in eval dataset is 249


In [102]:
# DO NOT PRODUCE A TEST DATASET FOR SLDB, AS INFERENCE PROCESS IS NOT DIRECT
# (IT IS ITERATIVE OVER UNSEEN DATA TIME SERIES, ALREADY PERSISTED AS A PICKLE FILE)

# on each customer dataset, adjust the number of examples to the number of training cores
for stage in ['train', 'eval']:
    # how many examples/rows must be removed from examples[stage] to comply with the number of cores
    examples_to_remove = len(examples[stage])%num_cores

    # remove the last 'examples_to_remove' examples from the dataset
    for _ in np.arange(examples_to_remove):
        examples[stage].pop(-1)


    # keep a record of the number of training and evaluation examples
    count[customer_id][stage] = len(examples[stage])
    print('For {} cores in Cloud TPU, the number of {} examples for {} was adjusted to {}'.\
         format(num_cores, stage, customer_id, len(examples[stage])))


For 8 cores in Cloud TPU, the number of train examples for MT_223 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_223 was adjusted to 248


In [103]:
# serialize the rows in examples['train'] and, if present, examples['eval']
# process each customer, then release data structures to avoid excesive memory consumption

# write a TFRecord file for each consumer_id/stage
for stage in ['train', 'eval']:
    # N_ROWS = sldb['stats'][stage]['n_rows']
    N_ROWS = len(examples[stage])
    filename = '{}/{}/{}.tfrecord'.format(sldb_dir, stage, customer_id)

    with tf.io.TFRecordWriter(filename) as writer:
        for row in np.arange(N_ROWS):

            example = tf.train.Example(
                # features within the example
                features=tf.train.Features(
                    # individual feature definition
                    feature={'encoder_input':
                             _float_feature_from_list_of_values(
                                 examples[stage][row]['encoder_input']),
                             'decoder_input':
                             _float_feature_from_list_of_values(
                                 examples[stage][row]['decoder_input']),
                             'target':
                             _float_feature_from_list_of_values(
                                 examples[stage][row]['target']),
                             'id':
                             _float_feature_from_list_of_values(
                                 examples[stage][row]['id'])
                             }
                )
            )
            serialized_example = example.SerializeToString()
            writer.write(serialized_example)

        # report TFRecord file as completed
        print('Persisted {} TFRecord file for {}'.format(stage, customer_id))

Persisted train TFRecord file for MT_223
Persisted eval TFRecord file for MT_223


In [104]:
count

{'MT_223': {'train': 4912, 'eval': 248}}

In [105]:
# SLDB is now updated to 370 elements with all zero-value lectures for MT-223

In [106]:
# update the example_count dataframe

In [107]:
example_count_df = pd.read_pickle('/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX/example_count.pkl')

In [108]:
example_count_df

,train
customer_id,
MT_001,4912
MT_002,4912
MT_003,4912
MT_004,4912
MT_005,4912
...,...
MT_366,4912
MT_367,4912
MT_368,4912


In [109]:
# insert a row for MT-223
example_count_df.loc['MT_223'] = 4912

In [115]:
# sort the dataframe to align the new row
example_count_df = example_count_df.sort_index()

In [117]:
example_count_df.loc['MT_222':'MT_224']

,train
customer_id,
MT_222,4912
MT_223,4912
MT_224,4912


In [118]:
# get the new number of training examples, it should be equal to
1790512 + 4912

1795424

In [119]:
np.sum(example_count_df.loc['MT_001':'MT_370']['train'])

1795424